In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
dj.conn()

Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [19]:
stack = 'MD589'
yaml_file = 'shape_params.yaml'
scripts_dir = os.environ['REPO_DIR']

In [37]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    size_positive : int   #size of file
    size_negative : int   #size of file
    """
    bucket = "mousebrainatlas-data"
    credFiles = scripts_dir+'/Cell_datajoint/VaultBrain/credFiles.yaml'
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state
            s3_fp = (Structure & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+'CSHL_cells_dm/'+stack+'/'+ struc + '/'+stack+'_'+struc+'_'+state+'.pkl'
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [39]:
Process.drop()

`kui_diffusionmap`.`__process` (2 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [38]:
diffusion = Process()
diffusion.populate(reserve_jobs=True)


populating for  6N
usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: too few arguments
/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2019-05-24 03:42:46,006	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-24_03-42-46_12141/logs.
2019-05-24 03:42:46,113	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:29571 to respond...
2019-05-24 03:42:46,236	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:40265 to respond...
2019-05-24 03:42:46,239	INFO services.py:804 -- Starting Redis shard with 30.0 GB max memory.
2019-05-24 03:42:46,254	INFO node.py:483 -- Process ST


The user-provided path /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_dm/MD589/SNR/MD589_SNR_positive.pkl does not exist.
/Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_dm/MD589/SNR/MD589_SNR_positive.pkl s3://mousebrainatlas-data/CSHL_cells_dm/MD589/SNR/MD589_SNR_positive.pkl


NoSuchKey: NoSuchKey: message: The specified key does not exist.

In [16]:
diffusion.fetch(as_dict=True)

[OrderedDict([('id', 1), ('size_positive', 103), ('size_negative', 1615)])]